In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.autograd import Variable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
class DecoderRNN(nn.Module):
    def __init__(self, input_size, augmented_size, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.input_size = input_size
        self.augmented_size = augmented_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.cnn1 = nn.Conv1d(self.input_size, self.augmented_size//2 , kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.cnn2 = nn.Conv1d(self.augmented_size//2, self.augmented_size, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.lstm_1 = nn.LSTM(self.augmented_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_2 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_3 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_4 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_5 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_6 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        self.lstm_7 = nn.LSTM(self.hidden_size, self.hidden_size//2, num_layers=1, bidirectional=True)
        # map the output of LSTM to the output space
        self.out = nn.Linear(self.hidden_size, self.output_size)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        self.hidden1 = self.init_hidden()
        self.hidden2 = self.init_hidden()
        self.hidden3 = self.init_hidden()
        self.hidden4 = self.init_hidden()
        self.hidden5 = self.init_hidden()
        self.hidden6 = self.init_hidden()
        self.hidden7 = self.init_hidden()

        output = self.cnn1(input.view(-1,3,1))
        output = self.relu1(output)
        output = self.cnn2(output)
        output = self.relu2(output)

        output, self.hidden1 = self.lstm_1(output.view(-1,1,self.augmented_size), self.hidden1)
        output_1 = output

        output, self.hidden2 = self.lstm_2(output, self.hidden2)
        output_2 = output

        output, self.hidden3 = self.lstm_3(output + output_1, self.hidden3)  # skip_connection 1
        output_3 = output

        output, self.hidden4 = self.lstm_4(output + output_2, self.hidden4)  # skip_connection 2
        output_4 = output

        output, self.hidden5 = self.lstm_5(output + output_3, self.hidden5)  # skip_connection 3
        output_5 = output

        output, self.hidden6 = self.lstm_6(output + output_4, self.hidden6)  # skip_connection 4

        output, self.hidden7 = self.lstm_7(output + output_5, self.hidden7)  # skip_connection 5
        
        output = self.out(output).view(-1,1)
        # output = self.softmax(output)
        return output

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_size // 2, device=device),
                torch.randn(2, 1, self.hidden_size // 2, device=device))
print(torch.cuda.is_available())

True


In [2]:
import pickle

# load data from file

with open("/home/yiqin/2018summer_project/data/smooth_3d1s_augmented_small.pkl", "rb") as f:
    data= pickle.load(f)

train_X = []
train_Y = []
for i in range(len(data)):
    train_X.append(data[i][0])
    train_Y.append(data[i][1])
print(len(train_X))
print(len(train_Y))
    
with open("/home/yiqin/2018summer_project/data/smooth_3d_1start_val.pkl", "rb") as f:
    dic = pickle.load(f)
    val_X = dic["X"]
    val_Y = dic["Y"]

    
target_Tensor = train_Y
maximum_target = len(train_Y)


20610
20610


In [22]:
def penalty_loss(penalty, criterion, output, target):
    loss = 0
    for i in range(target.shape[0]):
        if int(target[i]) == 1:
            loss += penalty[0] * criterion(output[i], target[i])
        else:
            loss += penalty[1] * criterion(output[i], target[i])
    return loss

In [20]:
def train(input_tensor, target_tensor, decoder, decoder_optimizer, criterion, verbose = False, penalty = (1, 0.5)):
    decoder_optimizer.zero_grad()
    
    loss = 0
    
    
    decoder_output= decoder(input_tensor)
    
    if verbose:
        print("prediction score:", decoder_output.squeeze().detach().cpu().numpy())
        print("ground truch:", target_tensor.squeeze().cpu().numpy())

    loss += penalty_loss(penalty, criterion, decoder_output.squeeze(0), target_tensor.float())

    loss.backward()
    
    decoder_optimizer.step()
    

    return loss.item()

# In[63]:

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


# In[64]:

def trainIters(decoder, n_iters, print_every = 1000, plot_every = 100, learning_rate = 0.01, total_batch = maximum_target, gamma = 0.1):    
    start = time.time()
    
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    
    criterion = nn.MSELoss()
    
    scheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size = total_batch, gamma = gamma)
    
    
    for iter in range(1, n_iters + 1):
        num = iter % total_batch
        verbose = (iter % print_every == 0)
        input_tensor = train_X[num].to(device).float()
        target_tensor = target_Tensor[num].to(device).float()
        #input_tensor = Variable(input_tensor, requires_grad = True)
        #print(input_tensor.shape, target_tensor.shape)
        if input_tensor.shape[0]<2:
            continue
        if input_tensor.shape[0] != target_tensor.shape[0]:
            continue
        
        loss = train(input_tensor, target_tensor, decoder, 
                     decoder_optimizer, criterion, verbose = verbose)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            acc, one_acc = validate(decoder, val_X, val_Y)
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * print_every, print_loss_avg))
            
            print("validation accuracy:", acc)
            print("validation prediction accuracy:", one_acc)
            #torch.save(decoder.state_dict(), 'naive_lstm_train.pt')
        
        scheduler.step()
        

In [23]:
input_size = 3
augmented_size = 32
hidden_size = 256
output_size = 1

decoder = DecoderRNN(input_size,augmented_size, hidden_size, output_size).to(device)

trainIters(decoder, 10000, print_every=100, learning_rate=1e-2, gamma=0.2)



prediction score: [ 0.47167924  0.48142302  0.46412936  0.44353142  0.42820722  0.41869023
  0.41348806  0.41078737  0.40948361  0.40897501  0.40892646  0.40903223
  0.40908757  0.40912715  0.40920937  0.40935749  0.40957367  0.40971816
  0.4096607   0.40964043  0.40981945  0.41034183  0.41139561  0.4125596
  0.41347453  0.41400778  0.41421795  0.41428384  0.41423681  0.4141331
  0.41428113  0.41481674  0.41576254  0.41558644  0.40852177  0.41969401]
ground truch: [ 1.       0.5      0.25     0.125    0.25     0.5      1.       0.5      0.25
  0.125    0.25     0.5      1.       0.5      0.25     0.125    0.125
  0.25     0.5      1.       0.5      0.25     0.125    0.0625   0.03125
  0.03125  0.0625   0.125    0.25     0.5      1.       0.5      0.25
  0.125    0.25     0.5    ]
3380 223
0m 14s (- 24m 20s) (100 1%) 2.7196
validation accuracy: 1.7160%
validation prediction accuracy: 26.0090%
prediction score: [ 0.8457706   0.69059068  0.55562729  0.48181757  0.44551241  0.4294478
  0.4

3380 173
2m 19s (- 23m 32s) (900 9%) 2.5667
validation accuracy: 2.0710%
validation prediction accuracy: 40.4624%
prediction score: [ 0.85077548  0.60265303  0.46303695  0.39156091  0.36392054  0.35823703
  0.36136436  0.36687753  0.37195963  0.37553731  0.37751466  0.37836361
  0.37874538  0.37900448  0.3792322   0.37946141  0.37987578  0.38016656
  0.38033953  0.38031778  0.37998697  0.3796826   0.37984487  0.38068271
  0.38144886  0.38189968  0.38237751  0.38268265  0.3825562   0.38161501
  0.38033712  0.3793714   0.37903717  0.37943316  0.3808389   0.38261753
  0.38430306  0.38536665  0.38527131  0.3838827   0.38216358  0.38049376
  0.37828392  0.37496814  0.36952862  0.36129588  0.3500348   0.33642724
  0.32306036  0.31477445  0.3230918   0.36856484]
ground truch: [ 1.          0.5         0.25        0.125       0.0625      0.03125
  0.0625      0.125       0.25        0.5         1.          0.5         0.25
  0.125       0.0625      0.03125     0.015625    0.0078125   0.0039062

3380 123
4m 39s (- 22m 44s) (1700 17%) 2.5988
validation accuracy: 2.1598%
validation prediction accuracy: 59.3496%
prediction score: [ 0.93733943  0.57849407  0.38194954  0.30755109  0.30141252  0.32084489
  0.34597167  0.36917531  0.38691044  0.39809462  0.40475649  0.40851611
  0.41063601  0.41245508  0.41445595  0.41655433  0.41872868  0.42083934
  0.41994947  0.41682637  0.4124755   0.40596306  0.39565989  0.38022396
  0.36118615  0.34312171  0.34301257  0.36176378]
ground truch: [ 1.     0.5    0.25   0.5    1.     0.5    0.25   0.125  0.25   0.5    1.
  0.5    0.25   0.125  0.125  0.25   0.5    1.     0.5    0.25   0.125
  0.125  0.25   0.5    1.     0.5    0.25   0.5  ]
3380 1479
4m 57s (- 22m 35s) (1800 18%) 2.4356
validation accuracy: 7.0414%
validation prediction accuracy: 16.0920%
prediction score: [ 0.8170135   0.3938584   0.16828704  0.09050983  0.08890593  0.1132524
  0.13907188  0.15750349  0.16781187  0.17291862  0.17550528  0.17798787
  0.18088597  0.18382645  0.18667

3380 112
7m 41s (- 20m 47s) (2700 27%) 2.4222
validation accuracy: 2.1598%
validation prediction accuracy: 65.1786%
prediction score: [ 1.10554683  0.60838693  0.34550154  0.25649685  0.26149023  0.30341154
  0.34931296  0.38417426  0.40255198  0.40651232  0.40046993  0.38820931
  0.3742283   0.36246437  0.35561693  0.35344148  0.35558665  0.36177257
  0.36655465  0.36866811  0.36895803  0.36527735  0.36123374  0.36021411
  0.36113647  0.36265823  0.36387211  0.36503959  0.36638546  0.36220935
  0.34923786  0.32781613  0.29830962  0.25794685  0.21185869  0.18838543
  0.24491042  0.46625933]
ground truch: [ 1.       0.5      0.25     0.125    0.0625   0.0625   0.125    0.25     0.5
  1.       0.5      0.25     0.25     0.5      1.       0.5      0.25
  0.125    0.0625   0.03125  0.03125  0.0625   0.125    0.25     0.5      1.
  0.5      0.25     0.5      1.       0.5      0.25     0.125    0.0625
  0.0625   0.125    0.25     0.5    ]
3380 141
7m 59s (- 20m 33s) (2800 28%) 2.4067
validat

3380 275
9m 59s (- 18m 32s) (3500 35%) 2.2980
validation accuracy: 2.6923%
validation prediction accuracy: 33.0909%
prediction score: [ 0.89775598  0.50521493  0.25179952  0.19469684  0.25397742  0.3473393
  0.41434023  0.44880995  0.45821604  0.4549835   0.44854236  0.44105893
  0.43060014  0.41950074  0.40868506  0.40526888  0.41453537  0.43213886
  0.44366664  0.4420453   0.43420798  0.42994329  0.43463737  0.44883564
  0.46655211  0.48012862  0.47881067  0.46273759  0.43134427  0.38191947
  0.31955224  0.27967215  0.32385129  0.5026778 ]
ground truch: [ 1.       0.5      0.25     0.125    0.25     0.5      1.       0.5      0.25
  0.125    0.0625   0.125    0.25     0.5      1.       0.5      1.       0.5
  0.25     0.125    0.0625   0.03125  0.0625   0.125    0.25     0.5      1.
  0.5      0.25     0.125    0.0625   0.125    0.25     0.5    ]
3380 368
10m 17s (- 18m 17s) (3600 36%) 2.2807
validation accuracy: 3.5207%
validation prediction accuracy: 32.3370%
prediction score: [ 0.

3380 755
12m 41s (- 16m 8s) (4400 44%) 2.1265
validation accuracy: 7.4852%
validation prediction accuracy: 33.5099%
prediction score: [ 0.97144246  0.47878432  0.18338603  0.05852395  0.09712327  0.23881546
  0.40612176  0.55749983  0.63071167  0.59995234  0.51997769  0.45827037
  0.42942911  0.4241702   0.40118247  0.3236458   0.24729291  0.24082252
  0.29878956  0.35152277  0.35374433  0.32852885  0.30777219  0.33127975
  0.37695092  0.40723419  0.42055047  0.41459182  0.37515956  0.32134491
  0.31547409  0.38009298  0.47892535  0.55304074  0.5338434   0.42425591
  0.32880098  0.30542722  0.31581035  0.29231307  0.23896396  0.18379045
  0.17575759  0.28708819  0.61437118]
ground truch: [ 1.       0.5      0.25     0.125    0.0625   0.125    0.25     0.5      1.
  0.5      0.25     0.125    0.125    0.25     0.5      1.       0.5      0.25
  0.125    0.0625   0.03125  0.0625   0.125    0.25     0.5      1.       0.5
  0.25     0.125    0.0625   0.0625   0.125    0.25     0.5      1.  

3380 4689
15m 18s (- 13m 34s) (5300 53%) 1.7212
validation accuracy: 56.4793%
validation prediction accuracy: 40.7123%
prediction score: [ 0.83363658  0.54515141  0.23672312  0.07038039  0.03765076  0.11339697
  0.22826818  0.3139151   0.35539603  0.43166727  0.67214334  0.78860509
  0.55588663  0.3588444   0.25948426  0.18174985  0.15661594  0.18705976
  0.34862578  0.65367156  0.66941065  0.29816642  0.05740005  0.13559952
  0.40158916]
ground truch: [ 1.      0.5     0.5     1.      0.5     0.25    0.125   0.0625  0.125
  0.25    0.5     1.      0.5     0.25    0.125   0.0625  0.0625  0.125
  0.25    0.5     1.      0.5     0.25    0.25    0.5   ]
3380 7093
15m 36s (- 13m 17s) (5400 54%) 1.5122
validation accuracy: 70.1775%
validation prediction accuracy: 33.4414%
prediction score: [ 0.99326032  0.62790197  0.37621951  0.27282527  0.32381177  0.48143372
  0.62307554  0.64624399  0.55559707  0.42451602  0.37272143  0.46352726
  0.55366176  0.53369081  0.51799381  0.52494037  0.511405

3380 7666
18m 0s (- 11m 2s) (6200 62%) 1.4520
validation accuracy: 83.2249%
validation prediction accuracy: 36.6945%
prediction score: [ 0.99979687  0.42926216  0.03638661  0.04569685  0.48355186  0.77794796
  0.39809245  0.1060887   0.13381234  0.42359102  0.67930353  0.44217175
  0.18614233  0.1101571   0.24592441  0.61732799  0.98770761  0.65283126
  0.19810548  0.08269614  0.19184959  0.28867117  0.3578791   0.5523237
  0.6866526   0.45571107  0.16591164  0.06056058  0.18284532  0.44738126]
ground truch: [ 1.     0.5    0.25   0.25   0.5    1.     0.5    0.25   0.25   0.5    1.
  0.5    0.25   0.125  0.25   0.5    1.     0.5    0.25   0.125  0.125
  0.25   0.5    1.     0.5    0.25   0.125  0.125  0.25   0.5  ]
3380 8915
18m 18s (- 10m 45s) (6300 63%) 1.3466
validation accuracy: 88.1953%
validation prediction accuracy: 33.4380%
prediction score: [ 0.77257293  0.44713253  0.25316653  0.25218832  0.31499383  0.19265524
  0.11322874  0.13156733  0.28024474  0.59460944  0.80435276  0.5

3380 5730
20m 59s (- 8m 9s) (7200 72%) 1.1618
validation accuracy: 76.8047%
validation prediction accuracy: 45.3054%
prediction score: [ 1.17989326  0.7423768   0.45103082  0.34073949  0.35794863  0.39589602
  0.39681357  0.36960107  0.30247256  0.27312577  0.3759394   0.73372197
  0.5835498   0.57925069  0.32006851  0.43553585  0.58674061  0.89200342
  0.5486607   0.32808074  0.26322591  0.2593514   0.26092139  0.27265325
  0.28420612  0.35968143  0.66961366  1.17899179  0.75722176  0.44403633
  0.30246851  0.22247475  0.21242937  0.22413224  0.25738472  0.36508703
  0.66736096  1.10226655  0.76686138  0.5342806   0.43255228  0.33425519
  0.28228498  0.25822386  0.28029191  0.38294196  0.68532997]
ground truch: [ 1.       0.5      0.25     0.125    0.0625   0.03125  0.03125  0.0625
  0.125    0.25     0.5      1.       0.5      0.25     0.25     0.5      1.
  0.5      0.25     0.125    0.0625   0.03125  0.03125  0.0625   0.125
  0.25     0.5      1.       0.5      0.25     0.125    0.

KeyboardInterrupt: 

In [3]:
def validate(decoder, val_x, val_y, val_threshold = 0.5):

    count = 0
    total = 0
    total_1 = 0
    for i in range(len(val_x)):
        X = val_x[i].to(device).float()
        result = decoder(X).squeeze().cpu().detach().numpy()
        Y = val_y[i].squeeze()
        gt = (Y == 1).astype(int)
        #pr = (result > 0.5).astype(int)
        pr = []
        pr.append(1)
        for j in range(1, len(result) - 1):
            if result[j] > val_threshold and result[j] > result[j-1] and result[j] > result[j+1]:
                pr.append(1)
            else:
                pr.append(0)
        pr.append(0)
        pr = np.array(pr)
        
        count += np.sum(gt * pr)
        total += np.sum(gt)
        total_1 += np.sum(pr)
        
    print(total, total_1)

    acc = str('%.4f'%((count / total * 100)) + "%")
    if total_1 == 0.0:
        one_acc = str('%.4f'%(0) + "%")
    else:
        one_acc = str('%.4f'%((count / total_1 * 100)) + "%")
    return acc, one_acc


In [4]:
def evaluate(model, val_x):
    prediction = []
    for i in range(len(val_x)):
        X = val_x[i].to(device).float()
        result = model(X).squeeze().cpu().detach().numpy()
        pr = []
        pr.append(1)
        for j in range(1, len(result) - 1):
            if result[j] > 0.5 and result[j] > result[j-1] and result[j] > result[j+1]:
                pr.append(1)
            else:
                pr.append(0)
        pr.append(0)
        prediction.append(np.array(pr))
    return prediction

In [12]:
input_size = 3
augmented_size = 32
hidden_size = 256
output_size = 1
model = DecoderRNN(input_size, augmented_size, hidden_size, output_size).to(device)
model.load_state_dict(torch.load("/home/yiqin/2018summer_project/saved_model/HallofFame/Bi-LSTM-CNN1.pt"))
prediction = evaluate(model, val_X)
acc, one_acc = validate(model, val_X,val_Y, 0.52)
print(acc, one_acc)

3380 3095
68.9053% 75.2504%


In [13]:
import pickle
f = open("/home/yiqin/2018summer_project/prediction/BiLSTM-CNN.pkl", "wb")
pickle.dump(prediction, f)

In [2]:
def factorize(data_X, data_Y, flag):
    if flag:
        new_X = []
        new_Y = []
        for i in range(len(data_X)):
            X, Y = data_X[i], data_Y[i]
            flag = []
            for loc, j in enumerate(Y):
                if j == 1:
                    flag.append(loc)
            prev = 0
            for j in range(4, len(flag), 5):
                new_X.append(X[prev:flag[j]])
                new_Y.append(torch.from_numpy(Y[prev:flag[j]]))
                prev = flag[j]
    return new_X, new_Y

In [6]:
from copy import deepcopy

class CrossValidator:
    def __init__(self, model, compute_acc, partition=5, decoder=None, batch_size=BATCH_SIZE, epochs=10, lr=1e-2, augment_data=True, print_every = 1000, plot_every = 100):
        self.model=model
        with open("/home/yiqin/2018summer_project/data/smooth_3d1s_augmented.pkl", "rb") as f:
            data= pickle.load(f)
            
        self.data_size=len(self.data_X)
        self.partition=partition
        self.decoder=decoder
        self.compute_acc=compute_acc
        self.train_X=[]
        self.train_Y=[]
        self.val_X=[]
        self.val_Y=[]
        self.precision_history=[]
        self.recall_history=[]
        self.loss_history=[]
        self.batch_size=batch_size
        self.epochs=epochs
        self.lr=lr
        self.augment_data_flag=augment_data
        self.print_every = print_every
        self.plot_every = plot_every
        
        
    def create_data(self, part):
        train_X=[]
        train_Y=[]
        val_X=[]
        val_Y=[]
        cut=int(self.data_size/self.partition)
        for i in range(self.data_size):
            if i<cut*part or i>=cut*(part+1):
                train_X.append(np.array(self.data_X[i]))
                train_Y.append(np.array(self.data_Y[i]))
            else:
                val_X.append(np.array(self.data_X[i]))
                val_Y.append(np.array(self.data_Y[i]))
        return train_X, train_Y, val_X, val_Y
    
    def tensorize(self, p):
        p=np.array(p)
        p=torch.from_numpy(p).float()
        p=p.to(device)
        return p
    
    def compute(self):
        for i in range(self.partition):
            start = time.time()
            temptrain_X, temptrain_Y, tempval_X, tempval_Y = self.create_data(i)
            self.train_X, self.train_Y = factorize(temptrain_X, temptrain_Y)
            self.val_X, self.val_Y = factorize(tempval_X, tempval_Y)
            print(i, "phase 1 completed.")
            
            cur_model = deepcopy(self.model).to(device)
            
            optimizer = optim.SGD(cur_model.parameters(), lr = self.lr)
    
            criterion = nn.MSELoss()

            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = gamma)

            print_loss_total = 0
            plot_loss_total = 0
            
            for j in range(self.epochs):
                print("epoch %i"%j)
                for num in range(len(self.train_X)):
                    input_tensor = train_X[num].to(device).float()
                    target_tensor = target_Tensor[num].to(device).float()
                    
                    verbose = (num%1000) = 0
                    
                    loss = train(input_tensor, target_tensor, cur_model, 
                                 optimizer, criterion, verbose = verbose)
                    print_loss_total += loss
                    plot_loss_total += loss

                    if iter % self.print_every == 0:
                        acc, one_acc = validate(cur_model, self.val_X, self.val_Y)
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_total = 0
                        print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                                     iter, iter / n_iters * self.print_every, print_loss_avg))

                        print("validation accuracy:", acc)
                        print("validation prediction accuracy:", one_acc)
                        #torch.save(decoder.state_dict(), 'naive_lstm_train.pt')

                scheduler.step()


73
